In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_Sioux_simp.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_Sioux_simp.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_Sioux_simp.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_Sioux_simp_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_Sioux_simp.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("Sioux_simp");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/SiouxFalls_trips_simp.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 121 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  (1,2)   => 100.0
  (9,2)   => 200.0
  (5,1)   => 200.0
  (6,11)  => 400.0
  ⋮       => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 110 entries:
  "109" => {11,9}
  "29"  => {3,10}
  "81"  => {9,1}
  "54"  => {6,4}
  "32"  => {4,2}
  "101" => {11,1}
  "30"  => {3,11}
  "2"   => {1,3}
  "1"   => {1,2}
  "78"  => {8,9}
  "74"  => {8,4}
  "41"  => {5,1}
  "65"  => {7,5}
  "51"  => {6,1}
  "53"  => {6,3}
  "106" => {11,6}
  "27"  => {3,8}
  "75"  => {8,5}
  "42"  => {5,2}
  "33"  => {4,3}
  "28"  => {3,9}
  "50"  => {5,11}
  "52"  => {6,2}
  "63"  => {7,3}
  "92"  => {10,2}
  ⋮     => ⋮

In [8]:
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

110-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
 1300.0
  500.0
  100.0
  100.0
  200.0
    ⋮  
 2800.0
 4000.0
  500.0
  200.0
  300.0
 1500.0
  500.0
  400.0
  500.0
  800.0
 1400.0
 3900.0

In [10]:
for key=keys(odPairRoute)
    if contains(key, "110-")
        println(key)
    end
end

110-618
110-620
110-621
110-619
110-622


In [11]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 26
numRoute = 622
numOD = 110

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3998
Number of nonzeros in inequality constraint Jacobian.:      622
Number of nonzeros in Lagrangian Hessian.............:       26

Total number of variables............................:      648
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
getValue(linkFlow)

26-element Array{Float64,1}:
  1800.0
  3200.0
  1800.0
  3900.0
  3200.0
  4600.0
  4600.0
  9600.0
  4600.0
  9600.0
  7200.0
  7900.0
  3900.0
  7200.0
  6500.0
  5800.0
  6500.0
  5800.0
  4900.0
  7900.0
  4900.0
 12900.0
 12900.0
  5400.0
  4700.0
  5300.0

In [13]:
getObjectiveValue(m)

661254.0836640956

In [14]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numNodes = 11  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     8892
Number of nonzeros in inequality constraint Jacobian.:     3146
Number of nonzeros in Lagrangian Hessian.............:       26

Total number of variables............................:     3172
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1236
Total number of inequality constraints...............:     3146
        inequality constraints with only lower bounds:     3146
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [15]:
getValue(linkFlow)

26-element Array{Float64,1}:
  1800.0
  3200.0
  1800.0
  3900.0
  3200.0
  4600.0
  4600.0
  9600.0
  4700.0
  9600.0
  7200.0
  7900.0
  3900.0
  7200.0
  6500.0
  5800.0
  6500.0
  5800.0
  4900.0
  7900.0
  4900.0
 12900.0
 12900.0
  5300.0
  4600.0
  5400.0

In [16]:
getObjectiveValue(m)

661254.083610393

In [17]:
 1800.0
  3200.0
  1800.0
  3900.0
  3200.0
  4600.0
  4600.0
  9600.0
  4600.0
  9600.0
  7200.0
  7900.0
  3900.0
  7200.0
  6500.0
  5800.0
  6500.0
  5800.0
  4900.0
  7900.0
  4900.0
 12900.0
 12900.0
  5400.0
  4700.0
  5300.0

661254.0836640956

661254.0836640956